In [ ]:
from a_dataset_model import return_dataset, return_x_y, return_xgb_model
import pandas as pd
from datetime import timedelta

# 7. Prepare next hour input
dataset = return_dataset()
x, y = return_x_y(dataset)
model = return_xgb_model(x, y)

def next_hour_predict(dataset: pd.DataFrame, model):

    latest_timestamp = dataset['timestamp'].max()
    next_hour = latest_timestamp + pd.Timedelta(hours=1)

    # Create next-hour input for each device
    device_ids = dataset['device_id'].unique()
    next_hour_data = pd.DataFrame({
        'device_id': device_ids,
        'temperature': dataset['temperature'].iloc[-1],  # Assuming same temp as last known
        'hour': next_hour.hour,
        'dayofweek': next_hour.weekday(),
        'is_weekend': int(next_hour.weekday() >= 5),
        'is_holiday': 0,  # Update based on real holiday data if available
        'year': next_hour.year,
        'month': next_hour.month,
        'day': next_hour.day    
    })

    # 8. Predict next-hour power consumption
    feature_cols = ['device_id', 'temperature', 'hour', 'dayofweek', 'is_weekend', 'is_holiday', 'year', 'month', 'day']
    next_hour_input = next_hour_data[feature_cols]
    next_hour_predictions = model.predict(next_hour_input)
    next_hour_data['predicted_power_kwh'] = next_hour_predictions

    # 9. Display per device
    print(f"🔌 Predicted Power Consumption for {next_hour.strftime('%Y-%m-%d %H:%M:%S')}")
    print(next_hour_data[['device_id', 'predicted_power_kwh']])

    # 10. Total power consumption
    total_power = next_hour_data['predicted_power_kwh'].sum()
    print(f"\n🔋 Total Predicted Power Consumption for {next_hour.strftime('%Y-%m-%d %H:%M:%S')}: {total_power:.2f} kWh")

next_hour_predict(dataset, model)

KeyError: 'timestamp'

TypeError: cannot unpack non-iterable XGBRegressor object

In [58]:
# predict_power.py

import pandas as pd
from datetime import timedelta
from a_dataset_model import return_dataset, return_x_y, return_xgb_model

# 1. Load & clean your data
dataset = return_dataset()          # returns cleaned DataFrame
X, y = return_x_y(dataset)          # returns features & target

# 2. Train model & get scaler
#    Make sure return_xgb_model returns (model, scaler)
model, scaler = return_xgb_model(X, y)

def make_feature_df(ts: pd.Timestamp) -> pd.DataFrame:
    """
    Build a feature DataFrame for each device at timestamp ts.
    """
    devices = dataset['device_id'].unique()
    df = pd.DataFrame({'device_id': devices})
    df['temperature'] = dataset['temperature'].iloc[-1]        # last known temp
    df['hour']        = ts.hour
    df['dayofweek']   = ts.weekday()
    df['is_weekend']  = int(ts.weekday() >= 5)
    df['is_holiday']  = 0                                      # fill in using holidays lib if desired
    df['month']       = ts.month
    return df

def predict_at(ts: pd.Timestamp) -> pd.DataFrame:
    """
    Predict power_kwh per device at timestamp ts.
    """
    feat_df = make_feature_df(ts)
    feat_cols = ['device_id','temperature','hour','dayofweek','is_weekend','is_holiday','month']
    X_input = feat_df[feat_cols]
    # scale & predict
    X_scaled = scaler.transform(X_input)
    feat_df['predicted_power_kwh'] = model.predict(X_scaled)
    feat_df['timestamp'] = ts
    return feat_df

if __name__ == "__main__":
    last_ts = pd.to_datetime(dataset['timestamp'].max())
    
    # Next hour
    next_hr = last_ts + timedelta(hours=1)
    df_hr = predict_at(next_hr)
    print("=== Next Hour Predictions ===")
    print(df_hr[['device_id','predicted_power_kwh']])
    
    # Next day (24h ahead)
    next_day = last_ts + timedelta(days=1)
    df_day = predict_at(next_day)
    print("\n=== Next Day Predictions ===")
    print(df_day[['device_id','predicted_power_kwh']])
    
    # Next week (168h ahead)
    next_week = last_ts + timedelta(weeks=1)
    df_wk = predict_at(next_week)
    print("\n=== Next Week Predictions ===")
    print(df_wk[['device_id','predicted_power_kwh']])


TypeError: cannot unpack non-iterable XGBRegressor object